In [1]:
import requests #libreria per poter accedere ai siti web e prendere le informazioni che ci servono
import pandas as pd

In [2]:
url = "https://www.metacritic.com/news/xbox-game-pass-library" #sito web dove prendere i dati
headers = {"user-agent":"Safari/537.36"} #ci permette di collegarci al sito, come se stessimo utilizzando un browser.
req = requests.get(url,headers=headers) #collegamento al sitoweb passando come parametri: l'url e l'headers.
req #output della connessione, per capire se ci siamo collegati. Response 200 ci indica che siamo riusciti ad accedere.

<Response [200]>

In [3]:
req.text #codice hmtl della pagina

'<!doctype html>\n<html data-n-head-ssr lang="en" data-edition="us" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D,%22data-edition%22:%7B%22ssr%22:%22us%22%7D%7D">\n  <head >\n    <!-- running tag = \'metacritic.prod.079a10\' -->\n    <meta data-n-head="ssr" data-hid="charset" charset="utf-8"><meta data-n-head="ssr" data-hid="viewport" name="viewport" content="width=device-width, initial-scale=1"><meta data-n-head="ssr" data-hid="fb:admins" property="fb:admins" content="100001036810388"><meta data-n-head="ssr" data-hid="fb:app_id" property="fb:app_id" content="123113677890173"><meta data-n-head="ssr" data-hid="google-site-verification" name="google-site-verification" content="I1kHyfzmmG1fEVjq8GBUgkfCHc6PNtxce1_VyUuJhws"><meta data-n-head="ssr" data-hid="msapplication-navbutton-color" name="msapplication-navbutton-color" content="#424244"><meta data-n-head="ssr" data-hid="msapplication-square150x150logo" name="msapplication-square150x150logo" content="https://www.metacritic.com/a/m

In [4]:
from bs4 import BeautifulSoup

inspect = BeautifulSoup(req.text) #trasformiamo la variabile in un oggetto di BeautifulSoup, cosi da poter interagire con i tag del sito.
inspect

<!DOCTYPE html>
<html data-edition="us" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D,%22data-edition%22:%7B%22ssr%22:%22us%22%7D%7D" data-n-head-ssr="" lang="en">
<head>
<!-- running tag = 'metacritic.prod.079a10' -->
<meta charset="utf-8" data-hid="charset" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1" data-hid="viewport" data-n-head="ssr" name="viewport"/><meta content="100001036810388" data-hid="fb:admins" data-n-head="ssr" property="fb:admins"/><meta content="123113677890173" data-hid="fb:app_id" data-n-head="ssr" property="fb:app_id"/><meta content="I1kHyfzmmG1fEVjq8GBUgkfCHc6PNtxce1_VyUuJhws" data-hid="google-site-verification" data-n-head="ssr" name="google-site-verification"/><meta content="#424244" data-hid="msapplication-navbutton-color" data-n-head="ssr" name="msapplication-navbutton-color"/><meta content="https://www.metacritic.com/a/mstile-150.png" data-hid="msapplication-square150x150logo" data-n-head="ssr" name="msapplication-square150x150

In [5]:
title_games = inspect.find_all("a", rel="follow")
title_games = title_games[44:]                            # per estrarre solo i titoli che mi servivano, altrimenti restituiva anche precedenti non utili

links = [link.get("href") for link in title_games]         # per estrarre i link alla pagina dei vari giochi
all_titles = [game.text.strip() for game in title_games]   # per estrarre il nome dei giochi
all_titles

['Against the Storm',
 'Age of Empires Def. Ed.',
 'Age of Empires II Def. Ed.',
 'Age of Empires IV: Anniversary Ed.',
 'Age of Mythology: Retold',
 'Alice: Madness Returns',
 'Aliens: Dark Descent',
 'All You Need Is Help',
 'Among Us',
 "Another Crab's Treasure",
 'Anthem',
 'Anuchard',
 'Ara: History Untold',
 'Arcade Paradise',
 'Ark: Survival Ascended',
 'ARK: Ultimate Survivor Ed.',
 'Army of Two',
 'Arx Fatalis',
 'As Dusk Falls',
 'Ashen',
 "Assassin's Creed Odyssey",
 "Assassin's Creed Origins",
 "Assassin's Creed Valhalla",
 'Astroneer',
 'Atlas Fallen',
 'Banjo Kazooie: N n B',
 'Banjo-Kazooie',
 'Banjo-Tooie',
 "The Bard's Tale ARPG: Remastered and Resnarkled",
 "The Bard's Tale IV: Director's Cut",
 "The Bard's Tale Trilogy",
 'Batman: Arkham Knight',
 'Battlefield 3',
 'Battlefield 4',
 'Battlefield 1 Revolution',
 'Battlefield 2042',
 'Battlefield Hardline',
 'Battlefield V',
 'Battletech',
 'Battletoads',
 'Bejeweled 2',
 'Bejeweled 3',
 'Ben 10: Power Trip',
 'The Big

In [6]:
gamepass_list = [] #lista per contenere i dizionari che contengono i singoli giochi.

for game in range (len(all_titles)): #loop per prendere ogni dato per ogni singolo gioco

    title_games_gamepass = all_titles[game]

    #entrare dentro la pagina di ogni videogioco:

    link_game = links[game]
    req_games = requests.get(link_game, headers=headers)
    req_games = BeautifulSoup(req_games.text)
    
    # prendere le informazioni:
    
    metascore = req_games.find("div", class_="c-siteReviewScore_background c-siteReviewScore_background-critic_medium")
    if metascore:
        metascore = metascore.text
    else:
        metascore = None

    metascore_reviews = req_games.find("span", class_="c-ScoreCard_reviewsTotal u-block")
    if metascore != "tbd":
        if metascore_reviews:
           metascore_reviews = metascore_reviews.text.replace("Based on ", "").replace(" Critic Reviews", "")
        else:
           metascore_reviews = None
    else: metascore_reviews = None
    
    user_score = req_games.find("div", class_="c-siteReviewScore_background c-siteReviewScore_background-user")
    if user_score:
        user_score = user_score.text
    else:
        user_score = None
    
    released_on = req_games.find("div", class_="c-gameDetails_ReleaseDate u-flexbox u-flexbox-row")
    if released_on:
        released_on = released_on.text[-12:] 
    else:
        released_on = None
    
    developer = req_games.find("div", class_="c-gameDetails_Developer u-flexbox u-flexbox-row")
    if developer:
        developer = developer.text[10:].strip()
    else:
        developer = None
    
    genres = req_games.find("a", class_="c-globalButton_container g-text-normal g-height-100 u-flexbox u-flexbox-alignCenter u-pointer u-flexbox-justifyCenter g-width-fit-content")
    if genres:
        genres = genres.text.strip()
    else:
        genres = None

    game_gamepass = {"Titolo":title_games_gamepass,
                     "Metascore":metascore,
                     "Metascore_Reviews":metascore_reviews,
                     "User_Score":user_score,
                     "Developer":developer,
                     "Genres":genres,
                     "Released_on":released_on}
    
    gamepass_list.append(game_gamepass)

gamepass_list


[{'Titolo': 'Against the Storm',
  'Metascore': '91',
  'Metascore_Reviews': '20',
  'User_Score': '8.0',
  'Developer': 'Eremite Games',
  'Genres': 'Management',
  'Released_on': ' Dec 8, 2023'},
 {'Titolo': 'Age of Empires Def. Ed.',
  'Metascore': '69',
  'Metascore_Reviews': '51',
  'User_Score': '6.8',
  'Developer': 'Forgotten Empires LLC',
  'Genres': 'Command RTS',
  'Released_on': 'Feb 20, 2018'},
 {'Titolo': 'Age of Empires II Def. Ed.',
  'Metascore': '84',
  'Metascore_Reviews': '34',
  'User_Score': '8.8',
  'Developer': 'Forgotten Empires LLC',
  'Genres': 'Command RTS',
  'Released_on': 'Nov 14, 2019'},
 {'Titolo': 'Age of Empires IV: Anniversary Ed.',
  'Metascore': '83',
  'Metascore_Reviews': '5',
  'User_Score': '8.7',
  'Developer': 'Relic Entertainment',
  'Genres': 'Command RTS',
  'Released_on': 'Aug 22, 2023'},
 {'Titolo': 'Age of Mythology: Retold',
  'Metascore': '83',
  'Metascore_Reviews': '58',
  'User_Score': '7.9',
  'Developer': "World's Edge",
  'Genre

In [7]:
df = pd.DataFrame(gamepass_list) #creazione dataframe
df

,Titolo,Metascore,Metascore_Reviews,User_Score,Developer,Genres,Released_on
0,Against the Storm,91,20,8.0,Eremite Games,Management,"Dec 8, 2023"
1,Age of Empires Def. Ed.,69,51,6.8,Forgotten Empires LLC,Command RTS,"Feb 20, 2018"
2,Age of Empires II Def. Ed.,84,34,8.8,Forgotten Empires LLC,Command RTS,"Nov 14, 2019"
3,Age of Empires IV: Anniversary Ed.,83,5,8.7,Relic Entertainment,Command RTS,"Aug 22, 2023"
4,Age of Mythology: Retold,83,58,7.9,World's Edge,Real-Time Strategy,"Sep 4, 2024"
...,...,...,...,...,...,...,...
518,The Yakuza Remastered Collection,83,23,7.9,Ryu ga Gotoku Studios,Compilation,"Aug 20, 2019"
519,Zombie Army 4: Dead War,72,60,6.2,Rebellion,Third Person Shooter,"Feb 4, 2020"
520,Zoo Tycoon: Ultimate Animal Collection,70,6,5.9,Asobo Studio,Tycoon,"Oct 31, 2017"
521,Zuma,77,16,7.0,Oberon Media,Matching Puzzle,"Nov 22, 2005"


In [11]:
# # Creazione documento ".csv"
df.to_csv('gamepass_data_.csv', index=False)